In [1]:
print("hello")

hello


In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "D:\Generative.AI\Medical_Chatboat_Generative-AI\Data\Medical_book.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()


In [5]:
#docs

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

texts_chunk = text_splitter.split_documents(docs)

In [4]:
len(texts_chunk)

42207

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [6]:
result = embeddings.embed_query("Hello, world!")
print(len(result))

1536


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec



import os
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.environ.get("PINECONE_API_KEY")



# Step 2: Initialize the Pinecone client
pc = Pinecone(api_key=pinecone_api_key)

# Step 3: Define index name
index_name = "medical-chatbot"

# Step 4: Create the index
pc.create_index(
    name=index_name,
    dimension=1536,  # --> set according to your embedding vector size
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


c:\Users\ASUS\anaconda3\envs\medical_chat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-wwanfk8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

## Our Vector Embeddings Store in PineconeDB

* It means jitne aapke text_chunk hai utne hi  Pinecone vector DB mai store honge like 42207

In [8]:
from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    index_name = index_name,
    embedding = embeddings
)

isting index

## Load the ex

In [9]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding=embeddings
)

In [10]:
docsearch

## Retriver-->

In [ ]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k" : 3})  ## it give you 3 relevant answer

In [14]:
result = retriever.invoke("what is Acne")

In [16]:
result

[Document(id='b96d8cca-8098-4410-a05f-08cc2332562d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 242.0, 'page_label': '243', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'D:\\Generative.AI\\Medical_Chatboat_Generative-AI\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='KEY TERMS\nAcne—A skin condition in which raised bumps,'),
 Document(id='dd55f479-334f-4e68-8798-58e690fe75d0', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 241.0, 'page_label': '242', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'D:\\Generative.AI\\Medical_Chatboat_Generative-AI\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='KEY TERMS\nAcne—A skin condition in which raised bumps,'),
 Document(id='8dee3eb2-67ae-451e-8364-a3e51982da0a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-1

##  I want to relevant/crucial output. So , now we connect my LLms model 

In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    " You are an assistant for question-answer tasks"
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer , say that you "
    " don't know. Use three sentence maximum and keep the "
    "answer concise. \n\n"
    "{context}"  
)


prompt_template = ChatPromptTemplate.from_messages(
    [

    ("system", "system_prompt"),
    ("human", "{context}")
]

)



## Chain

In [32]:
question_answer_chain = create_stuff_documents_chain(llm,prompt_template)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
response = rag_chain.invoke({"input" : "What is Acne .?"})
print(response["answer"])

 a skin condition that occurs when hair follicles become clogged with oil and dead skin cells. This can lead to the formation of whiteheads, blackheads, and pimples on the face, chest, and back. Acne is most common during puberty, but can also affect adults. It is not a serious health threat, but can be frustrating and may cause scarring if left untreated. Treatment options include topical creams, oral medications, and lifestyle changes.


In [38]:
response = rag_chain.invoke({"input" : "What is the harmfull affect of paractamol .?"})
print(response["answer"])

 can cause headaches, nausea, vomiting, and dizziness. Other potential side effects include skin rashes, hair loss, and changes in vision. In rare cases, pyrimethamine can also cause more serious side effects such as anemia, liver damage, and allergic reactions. It is important for individuals taking pyrimethamine to closely monitor their symptoms and report any concerning side effects to their healthcare provider.
